### Atualização do market data e da base de informações

In [1]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo
import numpy as np

import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import pytz

import sqlite3 as sql


C:\Users\paiva\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [16]:
##################################
## FUNCOES PARA BAIXAR MKT DATA ##
##################################

### BAIXAR DADOS DO MT5
def getData_mt5(tickers,inicio,fim):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        if not mt5.initialize('C:\\Program Files\\ModalMais MetaTrader 5\\terminal64.exe', 
                              server='Rico-Demo', login=3000265192, password="Wa6duJy3"):
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_  = inicio.split('-')
    inicio   = [int(element) for element in inicio_]
    fim_     = fim.split('-')
    fim      = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")

    stocks = pd.DataFrame()

    # obtendo o par de acoes com base nas datas enviadas nas strings, definindo o timezone para nao interfir com o tisp do mt5
    for ativo in tickers:
        stock = mt5.copy_rates_range(ativo, mt5.TIMEFRAME_D1, datetime(inicio[0],inicio[1],inicio[2],tzinfo=timezone),
                                     datetime(fim[0],fim[1],fim[2]))
        stock_df = pd.DataFrame(stock) # Transforma o output em DataFrame
        stock_df['ativo']=ativo        # Adiciona a coluna "Ativo" com o nome do ativo atual
 
        # Gera o df "stocks" acumulando os retornos de todos os ativos
        if len(stocks)>0:  
            stocks = pd.concat([stocks, stock_df]).reset_index(drop=True)
        else:
            stocks = stock_df
    
    # Tratamento da base acumulada para ter as colunas [ativo, date, open, high, low, close, volume]
    stocks['date'] = stocks['time'].apply(lambda x: date.fromtimestamp(x+21600)) #21600 pra corrigir o fuso no momento de converter tsp em str
    stocks.drop('time',axis=1,inplace = True)
    stocks.drop('tick_volume',axis=1,inplace = True)
    stocks.drop('spread',axis=1,inplace = True)
    stocks = stocks[['ativo','date','open','high','low','close','real_volume']]
    stocks.rename(columns={'real_volume':'volume'}, inplace=True)
    
    # fecha a conexão ao MetaTrader 5
    mt5.shutdown()

    return stocks

############################################
## FUNCOES RELACIONADAS AO BANCO DE DADOS ##
############################################

# Conectar no DB do SQLITE3
def connect(db):

    sqlite3_conn = None

    try:
        sqlite3_conn = sql.connect(db)
        return sqlite3_conn

    except sql.Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()
            
# Deletar tabela
def delete_table(db,table):

    try: 

        # Conecta ao Banco de Dados
        connection = connect(db)
        c = connection.cursor()

        c.execute("DROP TABLE ? ", table)

        connection.commit()
        connection.close()

    except sql.Error as e:
        print(e)

    finally:
        connection.close()


In [ ]:
##########################################
## Atualizar a tabela TBL_MARKETDATA_1D ##
##########################################
['ABEV3','AZUL4','B3SA3','BBAS3','BBDC3','BBDC4','BBSE3','BEEF3','BPAC11','BRAP4','BRDT3','BRFS3','BRKM5','BRML3','BTOW3']
tickers_ibov = ['CCRO3','CIEL3','CMIG4','COGN3','CPFE3','CRFB3','CSAN3','CSNA3','CVCB3','CYRE3','ECOR3','EGIE3','ELET3','ELET6','EMBR3']

#                 'ENBR3','ENGI11','EQTL3','FLRY3','GGBR4','GNDI3','GOAU4','GOLL4','HAPV3','HGTX3','HYPE3','IGTA3','IRBR3','ITSA4','ITUB4',
#                 'JBSS3','KLBN11','LAME4','LREN3','MGLU3','MRFG3','MRVE3','MULT3','NTCO3','PCAR3','PETR3','PETR4','QUAL3','RADL3','RAIL3',
#                 'RENT3','SANB11','SBSP3','SULA11','SUZB3','TAEE11','TIMP3','TOTS3','UGPA3','USIM5','VALE3','VIVT4','VVAR3','WEGE3','YDUQ3']


stocks = getData_mt5(tickers_ibov, '2018-1-1','2020-7-30')

print('Dados obtidos do mt5')
print(stocks)

try:

    # Conecta ao Banco de Dados
    connection = connect('db_market.dblite')
    c = connection.cursor()

    # Criando a tabela caso ela ainda tenha sido criada
    c.execute(""" CREATE TABLE IF NOT EXISTS tbl_marketdata_1d ( 
               id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
               ativo  TEXT NOT NULL, 
               date   TEXT NOT NULL, 
               open   INTEGER NOT NULL,  
               high   INTEGER NOT NULL,  
               low    INTEGER NOT NULL,  
               close  INTEGER NOT NULL,  
               volume INTEGER); """)
    

    # Primeiramente, mostra os ultimos regitros da tabela tbl_market_data_1d
    # para revisar a data da base que será appendada nela
    df = pd.read_sql_query("""SELECT * FROM tbl_marketdata_1d 
                           ORDER BY id 
                           DESC limit 5;""", connection)

    print("Ultimos registros que ja haviam na tabela tbl_market_data_1d")
    print(df)

    continuar = input('continuar?')  # "sim" para continuar e appendar, ou qualquer outra coisa pra sair

    if (continuar == "sim"):
        # Inserir dados do DataFrame stock para a tabea tbl_marketdata_1d
        stocks.to_sql(name='tbl_marketdata_1d', con=connection, if_exists='append', index=False)

    connection.commit()
    connection.close()

except sql.Error as e:
    print(e)

finally:
    connection.close()


In [3]:
############################################
## Atualizar a tabela TBL_COMPOSICAO_IBOV ##
############################################

# Excel baixado do site da B3: http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm
comp_ibov = pd.read_excel(r"C:\Trading\LongShort\Banco de Dados\Carteira_Teorica_IBOV\2Q2020.xlsx")

comp_ibov['quarter'] = '2Q2020'

print(comp_ibov)

try: 
    # Conecta ao Banco de Dados
    connection = connect('db_market.dblite')
    c = connection.cursor()


    # Criando a tabela caso ela ainda tenha sido criada
    c.execute(""" CREATE TABLE IF NOT EXISTS tbl_composicao_ibov ( 
               id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
               quarter TEXT NOT NULL,    
               codigo  TEXT NOT NULL, 
               acao    TEXT NOT NULL,  
               tipo    TEXT NOT NULL,  
               qtd_teorica   BIGINT NOT NULL,  
               participacao  REAL NOT NULL); """)


    # Salvar o DataFrame comp_ibov como uma tabela SQL (tbl_composicao_ibov)
    comp_ibov.to_sql(name='tbl_composicao_ibov', con=connection, if_exists='append', index=False)

    connection.commit()
    connection.close()
    
except sql.Error as e:
    print(e)

finally:
    connection.close()


   codigo          acao      tipo  qtd_teorica  participacao quarter
0   ABEV3     AMBEV S/A        ON   4354228928         3.395  2Q2020
1   AZUL4          AZUL     PN N2    317471474         0.380  2Q2020
2   B3SA3            B3     ON NM   2046021644         5.405  2Q2020
3   BBAS3        BRASIL     ON NM   1418466803         2.780  2Q2020
4   BBDC3      BRADESCO     ON N1   1253093907         1.513  2Q2020
..    ...           ...       ...          ...           ...     ...
70  VALE3          VALE     ON NM   3292010807        10.155  2Q2020
71  VIVT4  TELEF BRASIL        PN    415131868         1.303  2Q2020
72  VVAR3     VIAVAREJO     ON NM   1146662628         0.724  2Q2020
73  WEGE3           WEG     ON NM    689271972         1.893  2Q2020
74  YDUQ3    YDUQS PART  ON ED NM    299667897         0.625  2Q2020

[75 rows x 6 columns]


In [ ]:
df = pd.read_sql("select * from daily_flights", conn)
df["delay_minutes"] = None
df.to_sql("daily_flights", conn, if_exists="replace")

- le o sql e salva em um dataframe
- inclui uma coluna no DF
- transforma o DF em uma tabela SQL


In [7]:
try: 
    
    # Conecta ao Banco de Dados
    connection = connect('db_market.dblite')
    c = connection.cursor()


    c.execute(""" DROP TABLE tbl_marketdata_1d """)

    # Primeiramente, mostra os ultimos regitros da tabela tbl_market_data_1d
    # para revisar a data da base que será appendada nela
    df = pd.read_sql_query("""SELECT * FROM tbl_marketdata_1d 
                           ORDER BY id 
                           DESC limit 5;""", connection)

    connection.commit()
    connection.close()

    
except sql.Error as e:
    print(e)

finally:
    connection.close()


no such table: tbl_marketdata_1d


In [17]:
delete_table('db_market.dblite','tbl_marketdata_1d')

near "?": syntax error
